In [ ]:
from coreLib.dataset import DataSet
from coreLib.utils import create_dir,LOG_INFO
data_dir= "/media/ansary/DriveData/Work/bengalAI/datasets/CraftData/source"
save_dir       = "/media/ansary/DriveData/Work/bengalAI/datasets/CraftData/"

#save_dir       = "/home/apsisdev/ansary/DATASETS/DETNEW"
#data_dir       = "/home/apsisdev/ansary/DATASETS/synthdata_source"

ds=DataSet(data_dir)


# #create dirs
save_dir   =  create_dir(save_dir,"synth")
imgs_dir   =  create_dir(save_dir,"imgs")
charmap_dir  =  create_dir(save_dir,"charmaps")
wordmap_dir  =  create_dir(save_dir,"wordmaps")



In [ ]:
from coreLib.render import createSceneImage,backgroundGenerator,createImageData
from coreLib.config import config
backGen=backgroundGenerator(ds,dim=(config.back_dim,config.back_dim))
back=next(backGen)

In [ ]:
#--------------------
# imports
#--------------------
import cv2
import numpy as np
import matplotlib.pyplot as plt
#--------------------
# format
#--------------------

#     #return char_mask#,word_mask

In [ ]:
from coreLib.config import config
vars(config)
# config.min_num_lines=1
# config.max_num_lines=10
# config.comp_dim=32
# config.min_line_height=4
# config.word_min_space=5
# config.word_max_space=10
# config.vert_max_space=10

In [ ]:
import os
import cv2
from tqdm.auto import tqdm

def saveData(nb):
    '''
        number of images to save
    '''
    for i in tqdm(range(nb)):
        try:
            # data execution
            page,labels=createSceneImage(ds)
            back=createImageData(backGen,page,labels)
            charmap,wordmap=get_targets(page,labels)
            # data formation
            img_path =os.path.join(imgs_dir,f"img{i}.png")
            char_path=os.path.join(charmap_dir,f"img{i}.png")
            word_path=os.path.join(wordmap_dir,f"img{i}.png")
            # save
            cv2.imwrite(img_path,cv2.resize(back,(256,256)))
            cv2.imwrite(char_path,cv2.resize(charmap,(256,256)))
            cv2.imwrite(word_path,cv2.resize(wordmap,(256,256)))
        except Exception as e:
            pass

In [ ]:
# saveData(10)

In [ ]:
page,labels=createSceneImage(ds)
back=createImageData(backGen,page,labels)
charmap,wordmap=get_targets(page,labels)


In [ ]:
textmap=cv2.resize(charmap,(256,256))
linkmap=cv2.resize(wordmap,(256,256))
plt.imshow(linkmap)
plt.show()

In [ ]:
boxes=[]
# get shape
img_h, img_w = textmap.shape
# get thresholded scores
_, text_score = cv2.threshold(textmap,thresh=0.5,maxval=1,type=cv2.THRESH_BINARY)
_, link_score = cv2.threshold(linkmap,thresh=0.5,maxval=1,type=cv2.THRESH_BINARY)
# get connected components
n_components, labels, stats, _ = cv2.connectedComponentsWithStats(np.clip(text_score + link_score, 0, 1).astype('uint8'),connectivity=4)
boxes = []
for component_id in range(1, n_components):
    # Filter by size
    size = stats[component_id, cv2.CC_STAT_AREA]

    if size < 10:
        continue

    # If the maximum value within this connected component is less than
    # text threshold, we skip it.
    if np.max(textmap[labels == component_id]) < 0.7:
        continue

    # Make segmentation map. It is 255 where we find text, 0 otherwise.
    segmap = np.zeros_like(textmap)
    segmap[labels == component_id] = 255
    segmap[np.logical_and(link_score, text_score)] = 0
#     x, y, w, h = [stats[component_id, key] for key in
#                     [cv2.CC_STAT_LEFT, cv2.CC_STAT_TOP, cv2.CC_STAT_WIDTH, cv2.CC_STAT_HEIGHT]]

#     # Expand the elements of the segmentation map
#     niter = int(np.sqrt(size * min(w, h) / (w * h)) * 2)
#     sx, sy = max(x - niter, 0), max(y - niter, 0)
#     ex, ey = min(x + w + niter + 1, img_w), min(y + h + niter + 1, img_h)
#     segmap[sy:ey, sx:ex] = cv2.dilate(
#         segmap[sy:ey, sx:ex],
#         cv2.getStructuringElement(cv2.MORPH_RECT, (1 + niter, 1 + niter)))
    plt.imshow(segmap)
    plt.show()
    segmap=cv2.resize(segmap,(1024,1024))
    idx = np.where(segmap>0)            
    y_min,y_max,x_min,x_max = np.min(idx[0]), np.max(idx[0]), np.min(idx[1]), np.max(idx[1])
    boxes.append([x_min,y_min,x_max,y_max])
len(boxes)

In [ ]:
for box in boxes:
    x_min,y_min,x_max,y_max=box
    cv2.rectangle(back,(x_min,y_min),(x_max,y_max),(255,0,0),2)
plt.imshow(back)

In [ ]:
cv2.imwrite("test.png",back)

In [ ]:
np.unique(page)

In [ ]:
labels

In [ ]:
page,labels=createSceneImage(ds)

In [ ]:
np.unique(page)

In [ ]:
labels

In [ ]:
from coreLib.render import padPage,processLine
iden=3
labels=[]
page_parts=[]
# select number of lines in an image
num_lines=random.randint(config.min_num_lines,config.max_num_lines)
for _ in range(num_lines):
    line_parts=[]
    line_labels=[]
    # select number of words
    num_words=random.randint(config.min_num_words,config.max_num_words)
    for _ in range(num_words):
        img,label,iden=create_word(iden=iden,
                        source_type="bangla",
                        data_type=random.choice(["handwritten","printed"]),
                        comp_type=random.choice(["number","grapheme"]),
                        ds=ds,
                        use_dict=random.choice([True,False]),
                        )
        line_labels.append(label)
        line_parts.append(img)


    # create the line image
    line_img=np.concatenate(line_parts,axis=1)
    line_img=processLine(line_img)
    # the page lines
    page_parts.append(line_img)
    labels.append(line_labels)





paded_parts=[]

for lidx,line_img in enumerate(page_parts):
    # pad lines 
    pad_height=random.randint(config.vert_min_space,config.vert_max_space)
    pad     =np.zeros((pad_height,config.back_dim))
    line_img=np.concatenate([line_img,pad],axis=0)
    paded_parts.append(line_img)

# page img
page=np.concatenate(paded_parts,axis=0)
page=padPage(page)

In [ ]:
plt.imshow(page)

In [ ]:
labels

In [ ]:
np.unique(page)